In [ ]:
!pip install catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np


from transformers import AutoTokenizer, AutoModel
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.metrics import precision_score, roc_auc_score

from sklearn.model_selection import StratifiedKFold


In [4]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

In [20]:
file_path = '/content/drive/MyDrive/biv hack/for-teams/payments_main.tsv'
data = pd.read_csv(file_path, sep='\t', header=None, names=['ID', 'Date', 'Price', 'Purpose', 'Category'])

data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')

data['Price'] = data['Price'].replace({',': '.', '-': '.'}, regex=True)
data['Price'] = pd.to_numeric(data['Price'], errors='coerce')
data['Price'] = data['Price'].apply(lambda x: float(x))

data = data.drop('ID', axis=1)

In [21]:
data.tail(3)

,Date,Price,Purpose,Category
24997,2024-11-07,1750000.0,Предоставление кредита по договору №Д-00803/03...,NaN
24998,2024-11-07,3560.0,Оплата гос. пошлины,NaN
24999,2024-11-07,2910.0,Оплата за Подушечки Любятово с шоколадной начи...,NaN


In [22]:
len(data)

25000

In [23]:
X = data[['Purpose', 'Price']]
y = data['Category']


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sergeyzh/rubert-tiny-turbo')
model = AutoModel.from_pretrained('sergeyzh/rubert-tiny-turbo')


In [28]:
from tqdm import tqdm

tqdm.pandas()

In [29]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [30]:
X['Purpose_Embedding'] = X['Purpose'].progress_apply(get_embeddings)
X_embeddings = np.vstack(X['Purpose_Embedding'].values)

# Combine embeddings and 'Price' column
X_final = np.hstack((X_embeddings, X[['Price']].values))

100%|██████████| 25000/25000 [05:31<00:00, 75.42it/s]
<ipython-input-30-78cd0c3538fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Purpose_Embedding'] = X['Purpose'].progress_apply(get_embeddings)


In [13]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [14]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_encoded),
    y=y_encoded
)
class_weights_list = class_weights.tolist()

In [15]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store metrics for each fold
precision_scores_lr, precision_scores_cb = [], []
roc_auc_scores_lr, roc_auc_scores_cb = [], []
classification_reports = []

In [16]:
for train_index, test_index in tqdm(skf.split(X_final, y_encoded), desc="Cross-Validation Progress", total=skf.get_n_splits()):
    # Split data
    X_train, X_test = X_final[train_index], X_final[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    # Logistic Regression
    lr = LogisticRegression(class_weight='balanced', max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    y_proba_lr = lr.predict_proba(X_test)

    # Calculate metrics for Logistic Regression
    precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
    roc_auc_lr = roc_auc_score(y_test, y_proba_lr, multi_class='ovr')
    precision_scores_lr.append(precision_lr)
    roc_auc_scores_lr.append(roc_auc_lr)

    # CatBoost Classifier
    cb = CatBoostClassifier(iterations=100, class_weights=class_weights_list, verbose=10)
    cb.fit(X_train, y_train)
    y_pred_cb = cb.predict(X_test)
    y_proba_cb = cb.predict_proba(X_test)

    # Calculate metrics for CatBoost
    precision_cb = precision_score(y_test, y_pred_cb, average='weighted')
    roc_auc_cb = roc_auc_score(y_test, y_proba_cb, multi_class='ovr')
    precision_scores_cb.append(precision_cb)
    roc_auc_scores_cb.append(roc_auc_cb)

    # Save classification report for this fold
    report = classification_report(y_test, y_pred_cb, target_names=le.classes_, output_dict=True)
    classification_reports.append(report)


Cross-Validation Progress:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.5
0:	learn: 1.7502309	total: 4.34s	remaining: 7m 9s
10:	learn: 0.4493693	total: 21.1s	remaining: 2m 50s
20:	learn: 0.1634305	total: 38.5s	remaining: 2m 24s
30:	learn: 0.0876322	total: 55.6s	remaining: 2m 3s
40:	learn: 0.0598022	total: 1m 12s	remaining: 1m 44s
50:	learn: 0.0439193	total: 1m 30s	remaining: 1m 26s
60:	learn: 0.0313569	total: 1m 47s	remaining: 1m 8s
70:	learn: 0.0247999	total: 2m 3s	remaining: 50.6s
80:	learn: 0.0207807	total: 2m 21s	remaining: 33.1s
90:	learn: 0.0169286	total: 2m 38s	remaining: 15.6s
99:	learn: 0.0153677	total: 2m 55s	remaining: 0us


Cross-Validation Progress:  20%|██        | 1/5 [03:01<12:07, 181.85s/it]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Learning rate set to 0.5
0:	learn: 1.7934819	total: 5.55s	remaining: 9m 10s
10:	learn: 0.4136386	total: 22.6s	remaining: 3m 2s
20:	learn: 0.1524751	total: 44.1s	remaining: 2m 45s
30:	learn: 0.0805027	total: 1m 4s	remaining: 2m 23s
40:	learn: 0.0524525	total: 1m 22s	remaining: 1m 58s
50:	learn: 0.0378354	total: 1m 44s	remaining: 1m 40s
60:	learn: 0.0300633	total: 2m 2s	remaining: 1m 18s
70:	learn: 0.0243260	total: 2m 19s	remaining: 57s
80:	learn: 0.0196060	total: 2m 37s	remaining: 36.8s
90:	learn: 0.0174239	total: 2m 54s	remaining: 17.2s
99:	learn: 0.0155137	total: 3m 9s	remaining: 0us


Cross-Validation Progress:  40%|████      | 2/5 [06:13<09:22, 187.66s/it]/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Cross-Validation Progress:  60%|██████    | 3/5 [09:13<06:08, 184.11s/it]/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.5
0:	learn: 1.7346385	total: 2.3s	remaining: 3m 47s
10:	learn: 0.4128732	total: 19.3s	remaining: 2m 35s
20:	learn: 0.1688039	total: 36.2s	remaining: 2m 16s
30:	learn: 0.0887858	total: 53.2s	remaining: 1m 58s
40:	learn: 0.0633248	total: 1m 9s	remaining: 1m 40s
50:	learn: 0.0464679	total: 1m 27s	remaining: 1m 23s
60:	learn: 0.0346247	total: 1m 44s	remaining: 1m 6s
70:	learn: 0.0287647	total: 2m 1s	remaining: 49.7s
80:	learn: 0.0227851	total: 2m 20s	remaining: 33s
90:	learn: 0.0187170	total: 2m 40s	remaining: 15.9s
99:	learn: 0.0164528	total: 2m 57s	remaining: 0us


Cross-Validation Progress:  80%|████████  | 4/5 [12:19<03:04, 184.68s/it]/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.5
0:	learn: 1.7541093	total: 2.33s	remaining: 3m 51s
10:	learn: 0.3945554	total: 19.6s	remaining: 2m 38s
20:	learn: 0.1491599	total: 40.3s	remaining: 2m 31s
30:	learn: 0.0784662	total: 57.8s	remaining: 2m 8s
40:	learn: 0.0524935	total: 1m 15s	remaining: 1m 48s
50:	learn: 0.0393793	total: 1m 33s	remaining: 1m 29s
60:	learn: 0.0299553	total: 1m 49s	remaining: 1m 10s
70:	learn: 0.0236547	total: 2m 6s	remaining: 51.7s
80:	learn: 0.0197357	total: 2m 23s	remaining: 33.7s
90:	learn: 0.0166906	total: 2m 40s	remaining: 15.9s
99:	learn: 0.0147157	total: 2m 56s	remaining: 0us


Cross-Validation Progress: 100%|██████████| 5/5 [15:20<00:00, 184.03s/it]


In [17]:
avg_precision_lr = np.mean(precision_scores_lr)
avg_roc_auc_lr = np.mean(roc_auc_scores_lr)
avg_precision_cb = np.mean(precision_scores_cb)
avg_roc_auc_cb = np.mean(roc_auc_scores_cb)

In [18]:
final_report = {}
for cls in le.classes_:
    cls_metrics = [report[cls] for report in classification_reports if cls in report]
    avg_cls_metrics = {
        metric: np.mean([cls_metric[metric] for cls_metric in cls_metrics])
        for metric in cls_metrics[0]
    }
    final_report[cls] = avg_cls_metrics

In [19]:
# Display results
print("Logistic Regression:")
print(f"Average Precision: {avg_precision_lr:.4f}")
print(f"Average ROC AUC: {avg_roc_auc_lr:.4f}\n")

print("CatBoost:")
print(f"Average Precision: {avg_precision_cb:.4f}")
print(f"Average ROC AUC: {avg_roc_auc_cb:.4f}\n")

print("Aggregated Classification Report:")
for cls, metrics in final_report.items():
    print(f"\nClass: {cls}")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

Logistic Regression:
Average Precision: 0.7128
Average ROC AUC: 0.9187

CatBoost:
Average Precision: 0.9655
Average ROC AUC: 0.9968

Aggregated Classification Report:

Class: BANK_SERVICE
  precision: 1.0000
  recall: 0.9200
  f1-score: 0.9542
  support: 9.8000

Class: FOOD_GOODS
  precision: 0.9477
  recall: 0.9556
  f1-score: 0.9507
  support: 18.0000

Class: LEASING
  precision: 0.9750
  recall: 1.0000
  f1-score: 0.9867
  support: 7.6000

Class: LOAN
  precision: 1.0000
  recall: 1.0000
  f1-score: 1.0000
  support: 8.2000

Class: NON_FOOD_GOODS
  precision: 0.9414
  recall: 0.9579
  f1-score: 0.9487
  support: 19.2000

Class: NOT_CLASSIFIED
  precision: 1.0000
  recall: 0.7000
  f1-score: 0.8071
  support: 4.6000

Class: REALE_STATE
  precision: 0.9714
  recall: 1.0000
  f1-score: 0.9846
  support: 5.4000

Class: SERVICE
  precision: 0.9603
  recall: 1.0000
  f1-score: 0.9789
  support: 17.6000

Class: TAX
  precision: 0.9667
  recall: 1.0000
  f1-score: 0.9818
  support: 9.6000
